# Data Preprocessing

Preprocessing consists of several steps:
- Loading csv-files with pandas
- separate testcases and deaths
- Remove entries with NaN/Empty Cells
- Remove irrelevant rows and columns
- Split year-month column into year and month column
- Adjust Geo-Isocode (Use 2 Letter ISO)
- Save cleaned csv-files

# Import Dependencies

In [100]:
import pandas as pd

# Load raw data
covid_data contains data on death/test cases of each eu-countries from 2020 to 2023.

pps_data contains data on local-adjusted gdp of each eu-countries from 2012 to 2023

In [101]:
filepath_to_unprocessed_data = '../data/unprocessed_data/'
covid_filename = 'covid_testcases_and_deaths_in_eu_2020_to_2023.csv'
pps_filename = 'cost_adjusted_gdp(ppp)_eu_2012_to_2023.csv'

covid_data = pd.read_csv(filepath_to_unprocessed_data + covid_filename)
pps_data = pd.read_csv(filepath_to_unprocessed_data + pps_filename)

# Preprocess covid death/test-cases file

split year_week column into year and week column

select only relevant columns

convert 3 letter iso to 2 letter iso country code.

In [102]:
covid_data[['year', 'week']] = covid_data['year_week'].str.split('-', n=1, expand=True)
covid_data['year'] = pd.to_numeric(covid_data['year'])
covid_data['week'] = pd.to_numeric(covid_data['week'])

covid_data = covid_data[['country', 'country_code', 'year', 'week', 'year_week', 'rate_14_day', 'indicator']]

country_iso_code_dict = {'AUT':'AT','BEL':'BE','BGR':'BG','HRV':'HR','CYP':'CY','DNK':'DK','EST':'EE','FIN':'FI','FRA':'FR',
                         'DEU':'DE','GRC':'GR','HUN':'HU','IRL':'IE','ITA':'IT','LVA':'LV','LTU':'LT','LUX':'LU','MLT':'MT',
                         'NLD':'NL','POL':'PL','PRT':'PT','ROU':'RO','SVK':'SK','SVN':'SI','ESP':'ES','SWE':'SE','GBR':'GB'}
covid_data = covid_data[covid_data['country_code'].isin(country_iso_code_dict.keys())] # remove unknown countries etc
covid_data['country_code'] = covid_data['country_code'].map(country_iso_code_dict)

# Split covid death/test-cases file

split covid data into death and case data

remove nan values

In [103]:
covid_deaths_data = covid_data[covid_data['indicator'] == 'deaths'].copy()
covid_deaths_data.drop(columns=['indicator'], inplace=True)
covid_deaths_data.dropna(inplace=True)
print(covid_deaths_data)

covid_cases_data = covid_data[covid_data['indicator'] == 'cases'].copy()
covid_cases_data.drop(columns=['indicator'], inplace=True)
covid_cases_data.dropna(inplace=True)
print(covid_cases_data)


       country country_code  year  week year_week  rate_14_day
213    Austria           AT  2020    10   2020-10     0.000000
214    Austria           AT  2020    11   2020-11     0.222744
215    Austria           AT  2020    12   2020-12     3.563899
216    Austria           AT  2020    13   2020-13    14.589713
217    Austria           AT  2020    14   2020-14    25.281412
...        ...          ...   ...   ...       ...          ...
12643   Sweden           SE  2023    43   2023-43    13.298475
12644   Sweden           SE  2023    44   2023-44    17.986427
12645   Sweden           SE  2023    45   2023-45    25.640226
12646   Sweden           SE  2023    46   2023-46    31.284903
12647   Sweden           SE  2023    47   2023-47    15.403270

[4873 rows x 6 columns]
       country country_code  year  week year_week  rate_14_day
9      Austria           AT  2020    10   2020-10     1.414423
10     Austria           AT  2020    11   2020-11    11.014677
11     Austria           AT  2

# Preprocess gdp file

select only relevant columns

rename column names to match covid dataset column names

In [104]:
pps_data = pps_data[['geo', 'TIME_PERIOD', 'OBS_VALUE']]
pps_data.rename(columns={'geo': 'country_code', 'TIME_PERIOD': 'year', 'OBS_VALUE':'pps'}, inplace=True)

print(pps_data)

    country_code  year  pps
0             AL  2012   30
1             AL  2013   29
2             AL  2014   30
3             AL  2015   30
4             AL  2016   30
..           ...   ...  ...
496           US  2018  140
497           US  2019  134
498           US  2020  135
499           US  2021  135
500           US  2022  134

[501 rows x 3 columns]


# Merge files

join datasets by year and country

In [105]:
combined_cases_data = pd.merge(covid_cases_data, pps_data, on=['country_code', 'year'], how='inner')
combined_death_data = pd.merge(covid_deaths_data, pps_data, on=['country_code', 'year'], how='inner')
combined_covid_data = pd.merge(covid_data, pps_data, on=['country_code', 'year'], how='inner')

# Save files

In [106]:
filepath_to_processed_data = '../data/processed_data/'

pps_data.to_csv(filepath_to_processed_data + "pps_data.csv", index=False)
covid_deaths_data.to_csv(filepath_to_processed_data + "covid_deaths_data.csv", index=False)
covid_cases_data.to_csv(filepath_to_processed_data + "covid_cases_data.csv", index=False)
combined_covid_data.to_csv(filepath_to_processed_data + "combined_covid_data.csv", index=False)
combined_cases_data.to_csv(filepath_to_processed_data + "combined_cases_data.csv", index=False)
combined_death_data.to_csv(filepath_to_processed_data + "combined_death_data.csv", index=False)